# Imports

In [1]:
from numpy.random import seed
seed(888)

from tensorflow import set_random_seed
set_random_seed(404)

In [2]:
import os
import numpy as np
import tensorflow as tf

from time import strftime
from PIL import Image

# Constants

In [3]:
# Shift tab for function defn
X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'

LOGGING_PATH = 'tensorboard_mnist_digit_logs/'

NUM_CLASSES = 10
VALIDATION_SIZE = 10000
TOTAL_INPUTS = 28*28*1

# Get the Data

In [4]:
%%time

y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)

CPU times: user 375 ms, sys: 7.82 ms, total: 382 ms
Wall time: 407 ms


In [5]:
y_train_all.shape

(60000,)

In [6]:
%%time

y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)

CPU times: user 66 ms, sys: 2.72 ms, total: 68.7 ms
Wall time: 86.9 ms


In [7]:
y_test.shape

(10000,)

In [8]:
%%time

x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)

CPU times: user 49 s, sys: 1.5 s, total: 50.5 s
Wall time: 52.2 s


In [9]:
%%time

x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)

CPU times: user 7.65 s, sys: 156 ms, total: 7.8 s
Wall time: 7.87 s


# Exploring the Data

In [10]:
x_train_all.shape

(60000, 784)

In [11]:
x_train_all[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [12]:
# Value of a digit for every training example
y_train_all.shape

(60000,)

In [13]:
x_test.shape

(10000, 784)

In [14]:
y_test.shape

(10000,)

In [15]:
# First five digits of training data
y_train_all[:5]

array([5, 0, 4, 1, 9])

# Data Preprocessing

In [16]:
# Get all pixel values between 0 and 1
x_train_all, x_test = x_train_all/255.0, x_test/255.0

### Convert target values using one hot encoding

In [17]:
# Output is in a categorical form
values = y_train_all[:5]
# Array element indexing - ith row means value of ith digit = no of column 1
np.eye(10)[values]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [18]:
# An example of Element indexing
np.eye(10)[2]

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])

In [19]:
# One Hot Encoding
y_train_all = np.eye(NUM_CLASSES)[y_train_all]

In [20]:
y_train_all.shape

(60000, 10)

In [21]:
y_test = np.eye(NUM_CLASSES)[y_test]

In [22]:
y_test.shape

(10000, 10)

### Create Validation Data Set

In [23]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]

In [24]:
x_val.shape

(10000, 784)

In [25]:
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

In [26]:
x_train.shape

(50000, 784)

In [27]:
y_train.shape

(50000, 10)

# Setup Tensorflow Graph

In [28]:
# X and Y are tensors
# Willing to accept any no of samples as long as the other dimension is total_inp/num_classes
X = tf.placeholder(tf.float32, shape=(None, TOTAL_INPUTS), name='X')
Y = tf.placeholder(tf.float32, shape=(None, NUM_CLASSES), name='labels')

### Neural Network Architecture

#### Hyperparameters

In [29]:
num_epochs = 50
learning_rate = 1e-3

n_hidden1 = 512
n_hidden2 = 64

In [30]:
# with tf.name_scope('hidden_layer1'):
#     initial_w1 = tf.truncated_normal(shape=[TOTAL_INPUTS, n_hidden1], stddev=0.1, seed=42)
#     w1 = tf.Variable(initial_value=initial_w1, name='w1')

#     initial_b1 = tf.constant(value=0.0, shape=[n_hidden1])
#     b1 = tf.Variable(initial_value=initial_b1, name='b1')

#     layer1_in = tf.matmul(X, w1) + b1

#     layer1_out = tf.nn.relu(layer1_in)

In [31]:
# with tf.name_scope('hidden_layer2'):
#     initial_w2 = tf.truncated_normal(shape=[n_hidden1, n_hidden2], stddev=0.1, seed=42)
#     w2 = tf.Variable(initial_value=initial_w2, name='w2')

#     initial_b2 = tf.constant(value=0.0, shape=[n_hidden2])
#     b2 = tf.Variable(initial_value=initial_b2, name='b2')

#     layer2_in = tf.matmul(layer1_out, w2) + b2

#     layer2_out = tf.nn.relu(layer2_in)

In [32]:
# with tf.name_scope('output_layer'):
#     initial_w3 = tf.truncated_normal(shape=[n_hidden2, NUM_CLASSES], stddev=0.1, seed=42)
#     w3 = tf.Variable(initial_value=initial_w3, name='w3')

#     initial_b3 = tf.constant(value=0.0, shape=[NUM_CLASSES])
#     b3 = tf.Variable(initial_value=initial_b3, name='b3')

#     layer3_in = tf.matmul(layer2_out, w3) + b3
#     output = tf.nn.softmax(layer3_in)

In [33]:
# A general function
def setup_layer (input, weight_dim, bias_dim, name):
    with tf.name_scope(name):
         # Outputs random numbers from a truncated normal distribution
        initial_w = tf.truncated_normal(shape=weight_dim, stddev=0.1, seed=42)
        # A variable maintains the state in a graph across calls to run()
        w = tf.Variable(initial_value=initial_w, name='W')

         # Initialise bias to zero
        initial_b = tf.constant(value=0.0, shape=bias_dim)
        b = tf.Variable(initial_value=initial_b, name='B')

        layer_in = tf.matmul(input, w) + b
        
        if name=='out':
            layer_out = tf.nn.softmax(layer_in)
        else:
             layer_out = tf.nn.relu(layer_in)
                
        tf.summary.histogram('weights', w)
        tf.summary.histogram('biases', b)
        return layer_out       

In [34]:
layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1], 
                      bias_dim=[n_hidden1], name='layer_1')
# Can add a dropout layer if overfitting by layer_drop = tf.nn.dropout(layer_1, keep_prob=0.8)
# Change layer_2 input as layer_drop
layer_2 = setup_layer(layer_1, weight_dim=[n_hidden1, n_hidden2], 
                      bias_dim=[n_hidden2], name='layer_2')
output = setup_layer(layer_2, weight_dim=[n_hidden2, NUM_CLASSES], 
                     bias_dim=[NUM_CLASSES], name='out')

model_name = f'{n_hidden1}-{n_hidden2} LR{learning_rate} E{num_epochs}'

Instructions for updating:
Colocations handled automatically by placer.


# Tensorboard Setup

In [35]:
# Folder for Tensorboard
folder_name = f'{model_name} at {strftime("%H:%M")}'
directory = os.path.join(LOGGING_PATH, folder_name)
try:
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else:
    print('Succesfully created directories!')

Succesfully created directories!


# Loss, Optimisation and Metrics

### Defining Loss

In [36]:
with tf.name_scope('loss_calc'):
    # The parameters of reduce mean calculate softmax cross entropy loss between logits and labels
    # reduce_mean takes avg loss of all batches in which training is done
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output))

### Defining Optimiser

In [37]:
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    # Minimize minimises the loss and returns the training step to update variables(weights)
    train_step = optimizer.minimize(loss)

### Accuracy metric

In [38]:
with tf.name_scope('accuracy_calc'):
    # Equal makes a comparision
    # argmax as y has been encoded because of categories
    correct_pred = tf.equal(tf.argmax(output, axis=1), tf.argmax(Y, axis=1))
    # cast converts correct_pred to float
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [39]:
with tf.name_scope('performance'):
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('cost', loss)

### Check Input Image in Tensorboard

In [40]:
with tf.name_scope('show_image'):
    x_image = tf.reshape(X, [-1, 28, 28, 1])
    tf.summary.image('image_input', x_image, max_outputs=4)

# Run Session

In [41]:
# All calculations for the setup we've done take place inside a session
sess = tf.Session()

### Setup Filewriter and Merge Summaries

In [42]:
merged_summary = tf.summary.merge_all()

# Writer setup means telling it where to dump files
train_writer = tf.summary.FileWriter(directory + '/train')
train_writer.add_graph(sess.graph)

validation_writer = tf.summary.FileWriter(directory + '/validation/')
# We don't write add_graph as we want same graph for training writer and validation writer

### Initialise all variables

In [43]:
init = tf.global_variables_initializer()
sess.run(init)

In [44]:
# Now this variable has a weight
# w1.eval(sess)

In [45]:
# b3.eval(sess)

### Batching the data

In [46]:
size_of_batch = 1000
num_examples = y_train.shape[0]
num_iterations = int(num_examples/size_of_batch)
index_in_epoch = 0

In [47]:
def next_batch(batch_size, data, labels):
    global num_examples
    global index_in_epoch
    
    start = index_in_epoch
    index_in_epoch += batch_size
    if index_in_epoch > num_examples:
        start = 0
        index_in_epoch = batch_size
    end = index_in_epoch
    return data[start:end], labels[start:end]

### Training loop

In [48]:
for epoch in range(num_epochs):
    # =========For training dataset=================
    for i in range(num_iterations):
        batch_x, batch_y = next_batch(batch_size = size_of_batch, data = x_train, labels = y_train)
        
        # Creating feed dictionary to feed it into the sessions for the tensors created above
        feed_dictionary = {X:batch_x, Y:batch_y}
        
        # Running the calculation
        # Train step minimises loss on data provided via feed dict
        sess.run(train_step, feed_dict = feed_dictionary)
        
        batch_sum, batch_accuracy = sess.run(fetches = [merged_summary, accuracy], feed_dict = feed_dictionary)
        train_writer.add_summary(batch_sum, epoch)
    print(f'Epoch {epoch} \t| Training Accuracy = {batch_accuracy}')
    
    # =======For Validation dataset ==================
    # As we don't need to batch the validation data
    summary = sess.run(fetches = merged_summary, feed_dict={X:x_val, Y:y_val})
    validation_writer.add_summary(summary, epoch)
print('Done training')

Epoch 0 	| Training Accuracy = 0.8700000047683716
Epoch 1 	| Training Accuracy = 0.8859999775886536
Epoch 2 	| Training Accuracy = 0.9100000262260437
Epoch 3 	| Training Accuracy = 0.972000002861023
Epoch 4 	| Training Accuracy = 0.9769999980926514
Epoch 5 	| Training Accuracy = 0.9789999723434448
Epoch 6 	| Training Accuracy = 0.9789999723434448
Epoch 7 	| Training Accuracy = 0.9779999852180481
Epoch 8 	| Training Accuracy = 0.9860000014305115
Epoch 9 	| Training Accuracy = 0.9869999885559082
Epoch 10 	| Training Accuracy = 0.9879999756813049
Epoch 11 	| Training Accuracy = 0.9879999756813049
Epoch 12 	| Training Accuracy = 0.9890000224113464
Epoch 13 	| Training Accuracy = 0.9890000224113464
Epoch 14 	| Training Accuracy = 0.9890000224113464
Epoch 15 	| Training Accuracy = 0.9890000224113464
Epoch 16 	| Training Accuracy = 0.9900000095367432
Epoch 17 	| Training Accuracy = 0.9900000095367432
Epoch 18 	| Training Accuracy = 0.9900000095367432
Epoch 19 	| Training Accuracy = 0.99099999

# Making Predictions

In [49]:
img = Image.open('MNIST/test_img.png')
img

In [50]:
# Colour to black and white
bw = img.convert('L')
# Interchange black and white
img_array = np.invert(bw)
img_array.shape

(28, 28)

In [51]:
# Flatten the dimension
test_img = img_array.ravel()
test_img.shape

(784,)

In [52]:
prediction = sess.run(fetches=tf.argmax(output, axis=1), feed_dict={X:[test_img]})
prediction

array([2])

# Testing and Evaluation

In [53]:
test_accuracy = sess.run(fetches=accuracy, feed_dict={X:x_test, Y:y_test})
print(f'Test accuracy is {test_accuracy:0.2%}')

Test accuracy is 97.87%


# Reset for the Next Run

In [55]:
train_writer.close()
validation_writer.close()
sess.close()
tf.reset_default_graph()